<a id="section10"></a>
# <font color="#004D7F" size=5> 1.0. System setup</font>

In [1]:
!pip install lazypredict

In [2]:
import os
import lazypredict
import pandas as pd
import numpy as np

from google.colab import drive

np.random.seed(42)

drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/Máster UNED/TFM/Datasets/default_of_credit_card_clients.csv"

Mounted at /content/drive


<a id="section11"></a>
# <font color="#004D7F" size=5> 1.1. Read the dataset</font>

In [3]:
#Read CSV
df=pd.read_csv(dataset_path, delimiter=';')

df.pop('ID')

column_to_move = df.pop('default_payment_next_month')
df['default_payment_next_month'] = column_to_move

class_col = df.iloc[:,-1]
df = df.iloc[: , :-1]
df['default_payment_next_month'] = class_col
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default_payment_next_month
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,30000,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,80000,1,3,1,41,1,-1,0,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [4]:
df_x = df.drop('default_payment_next_month', axis = 1)
df_y = df['default_payment_next_month']

In [5]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_y = label_encoder.fit_transform(df_y)

labels = label_encoder.classes_

for label, integer_value in zip(labels, range(len(labels))):
    print(f"Label: {label} -> Integer Value: {integer_value}")

Label: 0 -> Integer Value: 0
Label: 1 -> Integer Value: 1


<a id="section11"></a>
# <font color="#004D7F" size=5> 1.2. Preparing experimentation</font>

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.30, random_state = 42,stratify=df_y)

In [7]:
df_train = pd.concat([X_train, pd.DataFrame({'default_payment_next_month': y_train})], axis = 1)
df_test = pd.concat([X_test, pd.DataFrame({'default_payment_next_month': y_test})], axis = 1)

In [ ]:
print(df_train['default_payment_next_month'].value_counts())
print(' ')
print(' ')
print(df_test['default_payment_next_month'].value_counts())
print(' ')

0.0    16355
1.0     4645
Name: default_payment_next_month, dtype: int64
 
 
0.0    7009
1.0    1991
Name: default_payment_next_month, dtype: int64
 


<a id="section21"></a>
# <font color="#004D7F" size=5> 2.1. Logistic Regression</font>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(solver='liblinear', max_iter=1000)
model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)

# Predict on the training set
y_pred_train = model.predict(X_train)

# Calculate training metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_auc = roc_auc_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)

# Calculate the log-loss for the training set
train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))


# Print the metrics
print(f"Test accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Train accuracy: {train_accuracy}")
print(f"Train AUC: {train_auc}")
print(f"Train precision: {train_precision}")
print(f"Train recall: {train_recall}")
print(f"Train loss: {train_loss}")

Test accuracy: 0.8083333333333333
Test AUC: 0.6034784221965029
Test precision: 0.6973293768545994
Test recall: 0.2360622802611753
Train accuracy: 0.8121904761904762
Train AUC: 0.6112954663400421
Train precision: 0.7148988350705089
Train recall: 0.2510226049515608
Train loss: 0.46299144379638796


<a id="section22"></a>
# <font color="#004D7F" size=5> 2.2. Gaussian Naive Bayes</font>

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

model = GaussianNB()

model.fit(X_train,y_train)
y_pred_test = model.predict(X_test)

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)

# Predict on the training set
y_pred_train = model.predict(X_train)

# Calculate training metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_auc = roc_auc_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)

# Calculate the log-loss for the training set
train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))


print(f"Test accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Train accuracy: {train_accuracy}")
print(f"Train AUC: {train_auc}")
print(f"Train precision: {train_precision}")
print(f"Train recall: {train_recall}")
print(f"Train loss: {train_loss}")

Test accuracy: 0.7335555555555555
Test AUC: 0.680784603622565
Test precision: 0.4257570229843123
Test recall: 0.5861376192867905
Train accuracy: 0.7460476190476191
Train AUC: 0.6996978371762946
Train precision: 0.4463840399002494
Train recall: 0.6165769644779333
Train loss: nan


<ipython-input-12-333d70e76da2>:25: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-12-333d70e76da2>:25: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-12-333d70e76da2>:25: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-12-333d70e76da2>:25: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))


<a id="section23"></a>
# <font color="#004D7F" size=5> 2.3. Decision Tree</font>

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

model = DecisionTreeClassifier()

model.fit(X_train,y_train)
y_pred_test = model.predict(X_test)

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)

# Predict on the training set
y_pred_train = model.predict(X_train)

# Calculate training metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_auc = roc_auc_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)

# Calculate the log-loss for the training set
train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))

# Predict on the validation set
y_pred_val = model.predict(X_val)

# Calculate validation metrics
validation_accuracy = accuracy_score(y_val, y_pred_val)
validation_auc = roc_auc_score(y_val, y_pred_val)
validation_precision = precision_score(y_val, y_pred_val)
validation_recall = recall_score(y_val, y_pred_val)

# Calculate the log-loss for the validation set
validation_loss = -np.mean(y_val * np.log(model.predict_proba(X_val)[:, 1]) + (1 - y_val) * np.log(1 - model.predict_proba(X_val)[:, 1]))

# Print the metrics
print(f"Test accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Train accuracy: {train_accuracy}")
print(f"Train AUC: {train_auc}")
print(f"Train precision: {train_precision}")
print(f"Train recall: {train_recall}")
print(f"Train loss: {train_loss}")
print(f"Validation accuracy: {validation_accuracy}")
print(f"Validation AUC: {validation_auc}")
print(f"Validation precision: {validation_precision}")
print(f"Validation recall: {validation_recall}")
print(f"Validation loss: {validation_loss}")

Test accuracy: 0.9423076923076923
Test AUC: 0.9433811802232855
Test precision: 0.9
Test recall: 0.9473684210526315
Train accuracy: 1.0
Train AUC: 1.0
Train precision: 1.0
Train recall: 1.0
Train loss: nan
Validation accuracy: 0.9243697478991597
Validation AUC: 0.9304804804804805
Validation precision: 0.86
Validation recall: 0.9555555555555556
Validation loss: nan


<ipython-input-32-cafa014a3ba8>:28: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-32-cafa014a3ba8>:28: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-32-cafa014a3ba8>:28: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-32-cafa014a3ba8>:28: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-32-cafa014a3ba8>:40: RuntimeWarning: divide by zero encounter

<a id="section24"></a>
# <font color="#004D7F" size=5> 2.4. Random Forest</font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

model = RandomForestClassifier()

parameters = {'n_estimators': [4, 6, 9, 10, 15],
              'max_features': ['log2', 'sqrt'],
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10],
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1, 5, 8]
             }

# Run the grid search
grid_obj = GridSearchCV(model, parameters)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the model to the best combination of parameters
model = grid_obj.best_estimator_

model.fit(X_train,y_train)
y_pred_test = model.predict(X_test)

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)

# Predict on the training set
y_pred_train = model.predict(X_train)

# Calculate training metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_auc = roc_auc_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)

# Calculate the log-loss for the training set
train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))

# Predict on the validation set
y_pred_val = model.predict(X_val)

# Calculate validation metrics
validation_accuracy = accuracy_score(y_val, y_pred_val)
validation_auc = roc_auc_score(y_val, y_pred_val)
validation_precision = precision_score(y_val, y_pred_val)
validation_recall = recall_score(y_val, y_pred_val)

# Calculate the log-loss for the validation set
validation_loss = -np.mean(y_val * np.log(model.predict_proba(X_val)[:, 1]) + (1 - y_val) * np.log(1 - model.predict_proba(X_val)[:, 1]))

# Print the metrics
print(f"Test accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Train accuracy: {train_accuracy}")
print(f"Train AUC: {train_auc}")
print(f"Train precision: {train_precision}")
print(f"Train recall: {train_recall}")
print(f"Train loss: {train_loss}")
print(f"Validation accuracy: {validation_accuracy}")
print(f"Validation AUC: {validation_auc}")
print(f"Validation precision: {validation_precision}")
print(f"Validation recall: {validation_recall}")
print(f"Validation loss: {validation_loss}")

Test accuracy: 0.9807692307692307
Test AUC: 0.9736842105263157
Test precision: 1.0
Test recall: 0.9473684210526315
Train accuracy: 0.9974874371859297
Train AUC: 0.9966216216216216
Train precision: 1.0
Train recall: 0.9932432432432432
Train loss: nan
Validation accuracy: 0.957983193277311
Validation AUC: 0.961861861861862
Validation precision: 0.9166666666666666
Validation recall: 0.9777777777777777
Validation loss: nan


<ipython-input-37-b3175f391dea>:41: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-37-b3175f391dea>:41: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-37-b3175f391dea>:41: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-37-b3175f391dea>:41: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-37-b3175f391dea>:53: RuntimeWarning: divide by zero encounter

<a id="section25"></a>
# <font color="#004D7F" size=5> 2.5. Support Vector Machine</font>

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
#X_val = scaler.transform(X_val)

model = svm.SVC(probability=True)

parameters = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

# Run the grid search
grid_obj = GridSearchCV(model, parameters)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the model to the best combination of parameters
model = grid_obj.best_estimator_

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)

# Predict on the training set
y_pred_train = model.predict(X_train)

# Calculate training metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_auc = roc_auc_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)

# Calculate the log-loss for the training set
train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))

# Predict on the validation set
#y_pred_val = model.predict(X_val)

# Calculate validation metrics
#validation_accuracy = accuracy_score(y_val, y_pred_val)
#validation_auc = roc_auc_score(y_val, y_pred_val)
#validation_precision = precision_score(y_val, y_pred_val)
#validation_recall = recall_score(y_val, y_pred_val)

# Calculate the log-loss for the validation set
#validation_loss = -np.mean(y_val * np.log(model.predict_proba(X_val)[:, 1]) + (1 - y_val) * np.log(1 - model.predict_proba(X_val)[:, 1]))

# Print the metrics
print(f"Test accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Train accuracy: {train_accuracy}")
print(f"Train AUC: {train_auc}")
print(f"Train precision: {train_precision}")
print(f"Train recall: {train_recall}")
print(f"Train loss: {train_loss}")
#print(f"Validation accuracy: {validation_accuracy}")
#print(f"Validation AUC: {validation_auc}")
#print(f"Validation precision: {validation_precision}")
#print(f"Validation recall: {validation_recall}")
#print(f"Validation loss: {validation_loss}")

<a id="section26"></a>
# <font color="#004D7F" size=5> 2.6. K-Nearest Neighbours</font>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

model = KNeighborsClassifier()

parameters = {'n_neighbors': [3, 4, 5, 10],
              'weights': ['uniform', 'distance'],
              'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'leaf_size' : [10, 20, 30, 50]
             }

# Run the grid search
grid_obj = GridSearchCV(model, parameters)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the model to the best combination of parameters
model = grid_obj.best_estimator_

model.fit(X_train,y_train)
y_pred_test = model.predict(X_test)

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_pred_test)
test_auc = roc_auc_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test)
test_recall = recall_score(y_test, y_pred_test)

# Predict on the training set
y_pred_train = model.predict(X_train)

# Calculate training metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_auc = roc_auc_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)

# Calculate the log-loss for the training set
train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))

# Print the metrics
print(f"Test accuracy: {test_accuracy}")
print(f"Test AUC: {test_auc}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Train accuracy: {train_accuracy}")
print(f"Train AUC: {train_auc}")
print(f"Train precision: {train_precision}")
print(f"Train recall: {train_recall}")
print(f"Train loss: {train_loss}")

<ipython-input-13-bb1cc267bcb3>:39: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-13-bb1cc267bcb3>:39: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))


Test accuracy: 0.8057777777777778
Test AUC: 0.6223341031216304
Test precision: 0.6313513513513513
Test recall: 0.2933199397287795
Train accuracy: 0.8265714285714286
Train AUC: 0.6508169947007972
Train precision: 0.7371158392434988
Train recall: 0.3356297093649085
Train loss: nan


<ipython-input-13-bb1cc267bcb3>:39: RuntimeWarning: divide by zero encountered in log
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))
<ipython-input-13-bb1cc267bcb3>:39: RuntimeWarning: invalid value encountered in multiply
  train_loss = -np.mean(y_train * np.log(model.predict_proba(X_train)[:, 1]) + (1 - y_train) * np.log(1 - model.predict_proba(X_train)[:, 1]))


<a id="section30"></a>
# <font color="#004D7F" size=5> 3.0. Lazy Predict</font>

In [8]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None, random_state=42)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)


 97%|█████████▋| 28/29 [02:42<00:07,  7.57s/it]

[LightGBM] [Info] Number of positive: 4645, number of negative: 16355
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003363 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3270
[LightGBM] [Info] Number of data points in the train set: 21000, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221190 -> initscore=-1.258742
[LightGBM] [Info] Start training from score -1.258742


100%|██████████| 29/29 [02:43<00:00,  5.65s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
GaussianNB                         0.73               0.68     0.68      0.75   
ExtraTreesClassifier               0.81               0.66     0.66      0.79   
LGBMClassifier                     0.82               0.65     0.65      0.80   
RandomForestClassifier             0.81               0.65     0.65      0.79   
XGBClassifier                      0.81               0.65     0.65      0.79   
SVC                                0.81               0.64     0.64      0.79   
NearestCentroid                    0.65               0.64     0.64      0.68   
KNeighborsClassifier               0.79               0.64     0.64      0.78   
QuadraticDiscriminantAnalysis      0.56               0.64     0.64      0.59   
AdaBoostClassifier                 0.81               0.63     0.63      0.79   
BaggingClassifier           

In [10]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
GaussianNB,0.73,0.68,0.68,0.75,0.08
ExtraTreesClassifier,0.81,0.66,0.66,0.79,3.98
LGBMClassifier,0.82,0.65,0.65,0.80,1.92
RandomForestClassifier,0.81,0.65,0.65,0.79,8.08
XGBClassifier,0.81,0.65,0.65,0.79,3.74
SVC,0.81,0.64,0.64,0.79,40.18
NearestCentroid,0.65,0.64,0.64,0.68,0.06
KNeighborsClassifier,0.79,0.64,0.64,0.78,2.38
QuadraticDiscriminantAnalysis,0.56,0.64,0.64,0.59,0.12
